For this notebook we will use a LangChain agent to query an AzureSQLDB with natural language.

We assume that we have an Azure SQL DB that is loaded with data, in this case tje Adventureworks sample data.
We also need an Azure OpenAI instance with a GPT-4 deployment.


In [6]:
import os
import pyodbc

server = '<USEYOUROWN>.database.windows.net'  
database = '<USEYOUROWN>'  
driver = 'ODBC Driver 18 for SQL Server'  
schema = 'SalesLT'

mysqlusername = '<USEYOUROWN>'  
mysqlpassword = '<USEYOUROWN>'
myazureopenaiendpoint = 'https://<USEYOUROWN>.openai.azure.com/'
myazureopenaikey = '<USEYOUROWN>'


In [7]:
cnxn = pyodbc.connect(f'DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={mysqlusername};PWD={mysqlpassword}')  
cursor = cnxn.cursor()  
print(cursor.execute(f'select * from {schema}.[Customer]'))
results = cursor.fetchall()

i = 0
for row in results:
    print(row)
    i += 1
    if i > 5: break 

(1, False, 'Mr.', 'Orlando', 'N.', 'Gee', None, 'A Bike Store', 'adventure-works\\pamela0', 'orlando0@adventure-works.com', '245-555-0173', 'L/Rlwxzp4w7RWmEgXX+/A7cXaePEPcp+KwQhl2fJL7w=', '1KjXYs4=', '3F5AE95E-B87D-4AED-95B4-C3797AFCB74F', datetime.datetime(2005, 8, 1, 0, 0))
(2, False, 'Mr.', 'Keith', None, 'Harris', None, 'Progressive Sports', 'adventure-works\\david8', 'keith0@adventure-works.com', '170-555-0127', 'YPdtRdvqeAhj6wyxEsFdshBDNXxkCXn+CRgbvJItknw=', 'fs1ZGhY=', 'E552F657-A9AF-4A7D-A645-C429D6E02491', datetime.datetime(2006, 8, 1, 0, 0))
(3, False, 'Ms.', 'Donna', 'F.', 'Carreras', None, 'Advanced Bike Components', 'adventure-works\\jillian0', 'donna0@adventure-works.com', '279-555-0130', 'LNoK27abGQo48gGue3EBV/UrlYSToV0/s87dCRV7uJk=', 'YTNH5Rw=', '130774B1-DB21-4EF3-98C8-C104BCD6ED6D', datetime.datetime(2005, 9, 1, 0, 0))
(4, False, 'Ms.', 'Janet', 'M.', 'Gates', None, 'Modular Cycle Systems', 'adventure-works\\jillian0', 'janet1@adventure-works.com', '710-555-0173', 'El

In [8]:
#from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
from langchain.sql_database import SQLDatabase

db_config = {  
    'drivername': 'mssql+pyodbc',  
    'username': f'{mysqlusername}@{server}', 
    'password': mysqlpassword,  
    'host': server,  
    'port': 1433,  
    'database': database,
    'query': {'driver': driver}  
}  

db_url = URL.create(**db_config)
db = SQLDatabase.from_uri(db_url, schema=f'{schema}')


In [9]:
from langchain.chat_models import AzureChatOpenAI

os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = myazureopenaiendpoint
os.environ["AZURE_OPENAI_API_KEY"] = myazureopenaikey

llm = AzureChatOpenAI(deployment_name="gpt-4", temperature=0, max_tokens=4000)

In [10]:
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [11]:
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.agents import create_sql_agent

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [12]:
agent_executor.run("how many tables are in the database?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Address, Customer, CustomerAddress, NewCustomer, Product, ProductCategory, ProductDescription, ProductModel, ProductModelProductDescription, SalesOrderDetail, SalesOrderHeader
Thought:I now know the final answer
Final Answer: There are 11 tables in the database.

> Finished chain.


'There are 11 tables in the database.'

In [13]:
agent_executor.run("what is the most expensive product?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Address, Customer, CustomerAddress, NewCustomer, Product, ProductCategory, ProductDescription, ProductModel, ProductModelProductDescription, SalesOrderDetail, SalesOrderHeader
Thought:The Product table seems to be the most relevant for this question. I should check its schema to see what columns it has.
Action: sql_db_schema
Action Input: Product
Observation: 
CREATE TABLE [SalesLT].[Product] (
	[ProductID] INTEGER NOT NULL IDENTITY(1,1), 
	[Name] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[ProductNumber] NVARCHAR(25) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Color] NVARCHAR(15) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[StandardCost] MONEY NOT NULL, 
	[ListPrice] MONEY NOT NULL, 
	[Size] NVARCHAR(5) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Weight] DECIMAL(8, 2) NULL, 
	[ProductCategoryID] INTEGER NULL, 
	[ProductModelID] INTEGER NULL, 
	[SellStartDate] DA

"The most expensive product is 'Road-150 Red, 62' with a price of 3578.27."

In [14]:
agent_executor.run("what is the most frequently sold product in the month of January?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Address, Customer, CustomerAddress, NewCustomer, Product, ProductCategory, ProductDescription, ProductModel, ProductModelProductDescription, SalesOrderDetail, SalesOrderHeader
Thought:The relevant tables for this query seem to be Product and SalesOrderDetail. I should check their schemas to understand their structure and the data they contain.
Action: sql_db_schema
Action Input: Product, SalesOrderDetail
Observation: 
CREATE TABLE [SalesLT].[Product] (
	[ProductID] INTEGER NOT NULL IDENTITY(1,1), 
	[Name] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[ProductNumber] NVARCHAR(25) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Color] NVARCHAR(15) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[StandardCost] MONEY NOT NULL, 
	[ListPrice] MONEY NOT NULL, 
	[Size] NVARCHAR(5) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Weight] DECIMAL(8, 2) NULL, 
	[ProductCategoryID] INTEGER 

"The most frequently sold product in the month of January is 'HL Road Frame - Black, 58' with a total of 120 units sold."